## **Introduction to ML for NLP [Network + Practical]**

### **CNN**

We have seen in notebook 6.A that we can correctly train a CNN in each language.

We now have to tune the model, and store the best results.

#### **Libraries**

We import the necessary libraries for the notebook.

In [1]:
# general
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

# pytorch
import torch

# reload libs
import importlib

# custom imports
import utility.models_pytorch as CustomPytorchModule

print("> Libraries Imported")

> Libraries Imported


#### **Setup**

- We set the device to *cuda*
- We import the dataset

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print("> Device:", device)

> Device: cuda


In [3]:
dataframe = pd.read_pickle("data/3_multi_eurlex_encoded.pkl")
dataframe.head(3)

,celex_id,labels,labels_new,text_en,text_de,text_it,text_pl,text_sv,text_en_enc,text_de_enc,text_it_enc,text_pl_enc,text_sv_enc,set
0,32010D0395,2,0,commission decision of december on state aid c...,beschluss der kommission vom dezember uber die...,decisione della commissione del dicembre conce...,decyzja komisji z dnia grudnia r w sprawie pom...,kommissionens beslut av den december om det st...,"[[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, ...","[[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, ...","[[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, ...","[[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, ...","[[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, ...",train
1,32012R0453,2,0,commission implementing regulation eu no of ma...,durchfuhrungsverordnung eu nr der kommission v...,regolamento di esecuzione ue n della commissio...,rozporzadzenie wykonawcze komisji ue nr z dnia...,kommissionens genomforandeforordning eu nr av ...,"[[2, 1275, 1276, 29, 100, 4, 743, 1277, 15, 12...","[[1302, 33, 1303, 3, 4, 5, 807, 15, 1304, 3, 6...","[[453, 10, 1422, 38, 14, 3, 4, 5, 990, 1423, 1...","[[1753, 1754, 3, 34, 24, 4, 5, 829, 7, 1755, 9...","[[2, 1239, 33, 23, 4, 5, 806, 7, 774, 4, 132, ...",train
2,32012D0043,2,0,commission implementing decision of january au...,durchfuhrungsbeschluss der kommission vom janu...,decisione di esecuzione della commissione del ...,decyzja wykonawcza komisji z dnia stycznia r u...,kommissionens genomforandebeslut av den januar...,"[[2, 1275, 3, 4, 1310, 1311, 15, 1015, 4, 1312...","[[1344, 3, 4, 5, 1345, 15, 1346, 74, 1347, 134...","[[2, 10, 1422, 3, 4, 5, 1454, 245, 1455, 24, 1...","[[2, 1791, 3, 4, 5, 1792, 7, 1, 1793, 1794, 65...","[[2, 1279, 4, 5, 1280, 7, 1281, 19, 1282, 1283...",train


#### **Create a Grid Search for Fine Tuning**

First, we setup the needed parameters.

In [4]:
# Parameters setup

COUNTS_EN = 3506
COUNTS_DE = 4216
COUNTS_IT = 4180
COUNTS_PL = 5255
COUNTS_SV = 4010

LEARNING_RATE_LIST = [0.001, 0.0001]
EMBEDDING_DIM_LIST = [1024, 2048]
KERNEL_SIZE_LIST = [3,5,7]
STRIDE_LIST = [1,3]
PADDING_LIST = [1,3]
DROPOUT_P_LIST = [0.0, 0.1]

Then, we execute a grid search.

In [5]:
iteration = 1

for LEARNING_RATE in LEARNING_RATE_LIST:
    for EMBEDDING_DIM in EMBEDDING_DIM_LIST:
        for KERNEL_SIZE in KERNEL_SIZE_LIST:
            for STRIDE in STRIDE_LIST:
                for PADDING in PADDING_LIST:
                    for DROPOUT_P in DROPOUT_P_LIST:

                        # re-import the class PytorchModel
                        # --> this is to be sure that each model is new, and not trained from the epoch of the previous one

                        importlib.reload(CustomPytorchModule)
                        PYTORCH_MODEL = CustomPytorchModule.PytorchModel

                        CNN_MODEL = PYTORCH_MODEL(

                            # set model and text language
                            model_type      = "CNN_fixed",
                            dataset         = dataframe,
                            language        = "en",

                            # set device, bacth size and epochs
                            device          = device,
                            batch_size      = 64,
                            epochs          = 50,

                            # set general hyperparameters
                            learning_rate   = LEARNING_RATE,

                            # set specific hyperparameters
                            vocab_size      = COUNTS_EN,
                            embedding_dim   = EMBEDDING_DIM,
                            out_channels    = 1,
                            kernel_size     = KERNEL_SIZE,
                            stride          = STRIDE,
                            padding         = PADDING,
                            dropout_p       = DROPOUT_P,
                        )


                        # train the model
                        CNN_MODEL.train_model()

                        iteration+=1

> Parameters imported for CNN_fixed
> Dataset correctly divided in training set, validation set and test set
> Created Pytorch datasets and dataloaders
> Initialization required 0.13 seconds
> Training Started
  - Total Epochs: 50


> Epoch 1: 100%|██████████| 60/60 [00:03<00:00, 18.74it/s]


 - Training Loss        0.9938
 - Validation Loss      0.9469
 - Validation Accuracy  0.5521

 - Validation Accuracy (per class)
   * Class 0	 0.5959 [202 out of 339]
   * Class 1	 0.1629 [51 out of 313]
   * Class 2	 0.8994 [277 out of 308]
   * Mean        0.5527

> ATTENTION: epoch 1 was the best one so far! The model has been saved :)



> Epoch 2: 100%|██████████| 60/60 [00:01<00:00, 56.94it/s]


 - Training Loss        0.8238
 - Validation Loss      0.8618
 - Validation Accuracy  0.6938

 - Validation Accuracy (per class)
   * Class 0	 0.5693 [193 out of 339]
   * Class 1	 0.9712 [304 out of 313]
   * Class 2	 0.5487 [169 out of 308]
   * Mean        0.6964

> ATTENTION: epoch 2 was the best one so far! The model has been saved :)



...

 - Training Loss        0.6107
 - Validation Loss      0.7525
 - Validation Accuracy  0.7792

 - Validation Accuracy (per class)
   * Class 0	 0.6755 [229 out of 339]
   * Class 1	 0.9137 [286 out of 313]
   * Class 2	 0.7565 [233 out of 308]
   * Mean        0.7819



> Epoch 49: 100%|██████████| 60/60 [00:01<00:00, 46.05it/s]


 - Training Loss        0.6078
 - Validation Loss      0.7626
 - Validation Accuracy  0.7813

 - Validation Accuracy (per class)
   * Class 0	 0.7021 [238 out of 339]
   * Class 1	 0.7348 [230 out of 313]
   * Class 2	 0.9156 [282 out of 308]
   * Mean        0.7842



> Epoch 50: 100%|██████████| 60/60 [00:01<00:00, 45.92it/s]


 - Training Loss        0.6098
 - Validation Loss      0.7593
 - Validation Accuracy  0.7969

 - Validation Accuracy (per class)
   * Class 0	 0.7817 [265 out of 339]
   * Class 1	 0.7636 [239 out of 313]
   * Class 2	 0.8474 [261 out of 308]
   * Mean        0.7976

